In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
import indycar.model.stint_simulator as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [192]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


In [193]:
df_oracle = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [194]:
df_oracle[df_oracle['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
11,12,31,3.0,5.0,2.0,1,1.0,-2.0,-1
38,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
86,12,93,0.0,0.0,0.0,0,0.0,0.0,0
117,12,128,0.0,0.0,0.0,0,0.0,0.0,0


In [195]:
stint.get_evalret(df_oracle)

pred: acc=0.6357615894039735, mae=3.4834437086092715,3.4834437086092715, rmse=27.496688741721854,r2=-0.41227034771091087, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.6357615894039735,
 3.4834437086092715,
 27.496688741721854,
 -0.41227034771091087)

### test

In [254]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df)


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
start pitlap: 7
simulation done: 32
start pitlap: 31
simulation done: 32
start pitlap: 33
simulation done: 32
start pitlap: 34
simulation done: 32
start pitlap: 35
simulation done: 32
start pitlap: 36
simulation done: 32
start pitlap: 39
simulation done: 32
start pitlap: 40
simulation done: 32
start pitlap: 41
simulation done: 32
start pitlap: 53
simulation done: 31
start pitlap: 54
simulation done: 31
start pitlap: 61
simulation done: 31
start pitlap: 63
simulation done: 31
start pitlap: 64
simulation done: 31
start pitlap: 65
simulation done: 31
start pitlap: 66
simulation done: 31
start pitlap: 67
simulation done: 31
start pitlap: 68
simulation done: 31
start pitlap: 69
simulation done: 31
start pitlap: 70
simulation done: 30
start pitlap: 71
simulation done: 30
start pitlap: 73
simulation done: 30
start pitlap: 74
simulation done: 30
start pitlap: 75

(0.6136363636363636,
 3.7954545454545454,
 33.76136363636363,
 0.05584104640119025)

In [ ]:
allpits, pitmat, maxlap = stint.get_pitlaps(verbose=True)
#print(len(allpits), maxlap)
nextpit, maxnext = stint.get_nextpit(pitmat, 30)

print(nextpit, maxnext)

In [ ]:
len(pitmat)

In [ ]:
pitmat

In [222]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,11.0,12.0,1.0,1,11.0,0.0,0
1,14,29,5.0,1.0,-4.0,-1,6.0,1.0,1
2,59,29,27.0,28.0,1.0,1,25.0,-2.0,-1
3,3,30,7.0,4.0,-3.0,-1,6.0,-1.0,-1
4,20,30,0.0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
146,25,156,18.0,0.0,-18.0,-1,14.0,-4.0,-1
147,64,156,6.0,3.0,-3.0,-1,5.0,-1.0,-1
148,59,157,21.0,19.0,-2.0,-1,19.0,-2.0,-1
149,60,157,16.0,1.0,-15.0,-1,3.0,-13.0,-1


In [221]:
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
11,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
38,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
86,12,93,0.0,0.0,0.0,0,0.0,0.0,0
117,12,128,0.0,0.0,0.0,0,0.0,0.0,0


In [90]:
correct = df[df['sign']==df['pred_sign']] 
acc = len(correct)/len(df)
acc

0.6291390728476821

In [94]:
stint.get_evalret(df)

pred: acc=0.6291390728476821, mae=3.4701986754966887,3.4701986754966887, rmse=27.576158940397352,r2=-0.5046232483344819, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.6291390728476821,
 3.4701986754966887,
 27.576158940397352,
 -0.5046232483344819)

In [98]:
ret ={ }
ret['oracle'] = df
ret['curtrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTCURTRACK )

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [99]:
stint.get_evalret(ret['curtrack'])

pred: acc=0.6026490066225165, mae=5.3841059602649,5.3841059602649, rmse=61.609271523178805,r2=-0.8693351182745332, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.6026490066225165, 5.3841059602649, 61.609271523178805, -0.8693351182745332)

In [101]:
ret['zerotrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTZERO )

start pitlap: 28
simulation done: 33
start pitlap: 29
simulation done: 33
start pitlap: 30
simulation done: 33
start pitlap: 31
simulation done: 33
start pitlap: 32
simulation done: 33
start pitlap: 33
simulation done: 33
start pitlap: 34
simulation done: 33
start pitlap: 35
simulation done: 33
start pitlap: 45
simulation done: 31
start pitlap: 49
simulation done: 31
start pitlap: 51
simulation done: 31
start pitlap: 52
simulation done: 31
start pitlap: 53
simulation done: 31
start pitlap: 59
simulation done: 30
start pitlap: 61
simulation done: 30
start pitlap: 69
simulation done: 29
start pitlap: 88
simulation done: 29
start pitlap: 89
simulation done: 29
start pitlap: 90
simulation done: 29
start pitlap: 91
simulation done: 29
start pitlap: 92
simulation done: 29
start pitlap: 93
simulation done: 29
start pitlap: 94
simulation done: 29
start pitlap: 95
simulation done: 29
start pitlap: 96
simulation done: 29
start pitlap: 97
simulation done: 29
start pitlap: 100
simulation done: 29


In [102]:
stint.get_evalret(ret['zerotrack'])

pred: acc=0.609271523178808, mae=6.006622516556291,6.006622516556291, rmse=71.2251655629139,r2=-0.6695263042106063, acc_naive=0.1456953642384106, mae_naive=4.529801324503311


(0.609271523178808, 6.006622516556291, 71.2251655629139, -0.6695263042106063)

### test sim by pred

In [248]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, datamode, loopcnt, featuremode = stint.FEATURE_STATUS):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init()
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    return b

In [214]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY, 3)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2


INFO:root:Using GPU


loading model...done!, ctx:gpu(0)
pred: acc=0.4557823129251701, mae=5.421768707482993,5.421768707482993, rmse=58.795918367346935,r2=-2.819438682563446, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.44594594594594594, mae=5.391891891891892,5.391891891891892, rmse=59.486486486486484,r2=-2.517986932339758, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.4391891891891892, mae=5.385135135135135,5.385135135135135, rmse=57.25,r2=-2.8189564597716563, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
[ 0.44697248  5.39959858 58.51080162 -2.71879402]


In [215]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,3)


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5238095238095238, mae=4.884353741496598,4.884353741496598, rmse=47.68707482993197,r2=-1.4224579319294186, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.4594594594594595, mae=4.986486486486487,4.986486486486487, rmse=50.54054054054054,r2=-1.8645951932432152, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.5, mae=5.006756756756757,5.006756756756757, rmse=51.317567567567565,r2=-1.7263753028807325, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
[ 0.49442299  4.95919899 49.84839431 -1.67114281]


In [216]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5306122448979592, mae=5.224489795918367,5.224489795918367, rmse=52.843537414965986,r2=-1.0284073953019082, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.5067567567567568, mae=5.236486486486487,5.236486486486487, rmse=53.817567567567565,r2=-1.1184119727421722, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.49324324324324326, mae=5.337837837837838,5.337837837837838, rmse=56.71621621621622,r2=-1.1775578564329625, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5135135135135135, mae=4.972972972972973,4.972972972972973, rmse=49.41891891891892,r2=-0.935519170628698, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.4797297297297297, mae=5.114864864864865,5.114864864864865, rmse=50.6554054054054,r2=-1.1197710495346098, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
[ 0.5047

In [218]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTCURTRACK,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTZERO,5)

INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5306122448979592, mae=5.340136054421769,5.340136054421769, rmse=49.51700680272109,r2=-0.8110458916838457, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.49324324324324326, mae=5.601351351351352,5.601351351351352, rmse=53.11486486486486,r2=-0.8403080076938652, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5067567567567568, mae=5.256756756756757,5.256756756756757, rmse=48.17567567567568,r2=-0.2609049446104692, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5337837837837838, mae=5.378378378378378,5.378378378378378, rmse=51.32432432432432,r2=-0.7125543262177982, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5135135135135135, mae=5.074324324324325,5.074324324324325, rmse=44.939189189189186,r2=-0.66436938643004, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
[ 0.51558

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5136986301369864, mae=5.602739726027397,5.602739726027397, rmse=63.013698630136986,r2=-1.048807125065398, acc_naive=0.136986301369863, mae_naive=4.6506849315068495
pred: acc=0.4527027027027027, mae=5.831081081081081,5.831081081081081, rmse=64.11486486486487,r2=-0.937313338124752, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.5067567567567568, mae=5.804054054054054,5.804054054054054, rmse=66.14189189189189,r2=-1.0214736894979843, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.47297297297297297, mae=5.851351351351352,5.851351351351352, rmse=64.77027027027027,r2=-1.1473624304508663, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.4527027027027027, mae=5.871621621621622,5.871621621621622, rmse=64.57432432432432,r2=-1.1207113438076775, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
[ 0.47976

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.4421768707482993, mae=8.17687074829932,8.17687074829932, rmse=105.8639455782313,r2=-0.5415562641681335, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.4864864864864865, mae=7.5675675675675675,7.5675675675675675, rmse=98.0945945945946,r2=-0.4329203067689229, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.4594594594594595, mae=7.878378378378378,7.878378378378378, rmse=97.63513513513513,r2=-0.3513376890899673, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.40816326530612246, mae=7.938775510204081,7.938775510204081, rmse=105.51700680272108,r2=-0.6142809809227268, acc_naive=0.1360544217687075, mae_naive=4.64625850340136
pred: acc=0.44594594594594594, mae=7.945945945945946,7.945945945945946, rmse=101.48648648648648,r2=-0.6073277532580967, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
[  0.448

In [249]:
accret = simulation('indy2013-2018-pitage-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5, featuremode = stint.FEATURE_PITAGE)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5102040816326531, mae=5.292517006802721,5.292517006802721, rmse=54.0,r2=-1.11524977612779, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.49324324324324326, mae=5.45945945945946,5.45945945945946, rmse=56.67567567567568,r2=-1.2326043129757949, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.49324324324324326, mae=5.3108108108108105,5.3108108108108105, rmse=55.7972972972973,r2=-1.2476409759398859, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.49324324324324326, mae=5.344594594594595,5.344594594594595, rmse=55.29054054054054,r2=-1.2211128635880129, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.4797297297297297, mae=5.452702702702703,5.452702702702703, rmse=58.62837837837838,r2=-1.2350791208638205, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
[ 0.49393271  5.372016

In [250]:
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.5238095238095238, mae=5.020408163265306,5.020408163265306, rmse=50.42176870748299,r2=-0.9224976356167864, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.5405405405405406, mae=5.344594594594595,5.344594594594595, rmse=56.57432432432432,r2=-0.928148986060175, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.5405405405405406, mae=5.087837837837838,5.087837837837838, rmse=53.1554054054054,r2=-0.9803346945856624, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.5405405405405406, mae=4.952702702702703,4.952702702702703, rmse=50.560810810810814,r2=-0.8848000381220142, acc_naive=0.13513513513513514, mae_naive=4.621621621621622
pred: acc=0.5436241610738255, mae=5.073825503355705,5.073825503355705, rmse=52.56375838926174,r2=-0.8118089086928848, acc_naive=0.1342281879194631, mae_naive=4.597315436241611
[ 0.5378110

In [251]:
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a

In [252]:
### pitmodel -- top8
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.564625850340136, mae=5.068027210884353,5.068027210884353, rmse=51.04081632653061,r2=-0.8634852247959852, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.5337837837837838, mae=5.141891891891892,5.141891891891892, rmse=51.38513513513514,r2=-0.7863332508046543, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5202702702702703, mae=5.081081081081081,5.081081081081081, rmse=50.445945945945944,r2=-0.8588041739352783, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5608108108108109, mae=4.864864864864865,4.864864864864865, rmse=49.25675675675676,r2=-0.8804017630727243, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5472972972972973, mae=4.993243243243243,4.993243243243243, rmse=51.277027027027025,r2=-0.9854692272881551, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
[ 0.5453

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a

In [255]:
### pitmodel -- top8
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.46938775510204084, mae=5.081632653061225,5.081632653061225, rmse=49.31292517006803,r2=-1.1418553045743436, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.4797297297297297, mae=5.101351351351352,5.101351351351352, rmse=48.91216216216216,r2=-1.0707162074403933, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5135135135135135, mae=4.864864864864865,4.864864864864865, rmse=49.810810810810814,r2=-1.063224857274149, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.47297297297297297, mae=5.277027027027027,5.277027027027027, rmse=53.33108108108108,r2=-1.08833414376019, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.4594594594594595, mae=5.202702702702703,5.202702702702703, rmse=55.027027027027025,r2=-1.2912106816915974, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
[ 0.4790

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a

### sim pred

In [247]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()

#stint._dataset_id = 'indy2013-2018'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

#stint._dataset_id = 'indy2013-2018-nocarid-context40'
#stint._feature_mode = stint.FEATURE_STATUS
stint._dataset_id = 'indy2013-2018-pitage-nocarid-context40'
stint._feature_mode = stint.FEATURE_PITAGE

stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df2 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
stint.get_evalret(df2)
df3 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df3)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
pred: acc=0.541095890410959, mae=5.2534246575342465,5.2534246575342465, rmse=54.294520547945204,r2=-1.2790757209360946, acc_naive=0.136986301369863, mae_naive=4.6506849315068495
pred: acc=0.5, mae=5.635135135135135,5.635135135135135, rmse=55.04054054054054,r2=-0.4424133223741278, acc_naive=0.13513513513513514, mae_naive=4.614864864864865


(0.5, 5.635135135135135, 55.04054054054054, -0.4424133223741278)

In [227]:
df2[df2['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
10,12,31,3.0,5.0,2.0,1,1.0,-2.0,-1
36,12,49,5.0,0.0,-5.0,-1,3.0,-2.0,-1
84,12,93,0.0,0.0,0.0,0,0.0,0.0,0
113,12,128,0.0,0.0,0.0,0,0.0,0.0,0


In [225]:
df2

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,11.0,12.0,1.0,1,14.0,3.0,1
1,14,29,5.0,1.0,-4.0,-1,6.0,1.0,1
2,59,29,27.0,28.0,1.0,1,24.0,-3.0,-1
3,3,30,7.0,4.0,-3.0,-1,8.0,1.0,1
4,20,30,0.0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
142,25,156,18.0,0.0,-18.0,-1,16.0,-2.0,-1
143,64,156,6.0,3.0,-3.0,-1,19.0,13.0,1
144,59,157,21.0,19.0,-2.0,-1,20.0,-1.0,-1
145,60,157,16.0,1.0,-15.0,-1,16.0,0.0,0


In [224]:
stint.get_evalret(df2)

pred: acc=0.48299319727891155, mae=5.129251700680272,5.129251700680272, rmse=51.7687074829932,r2=-1.1285052686056556, acc_naive=0.1360544217687075, mae_naive=4.625850340136054


(0.48299319727891155, 5.129251700680272, 51.7687074829932, -1.1285052686056556)

In [209]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
    
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

pred: acc=0.5170068027210885, mae=5.190476190476191,5.190476190476191, rmse=51.925170068027214,r2=-0.9451280397745003, acc_naive=0.1360544217687075, mae_naive=4.625850340136054
pred: acc=0.49324324324324326, mae=5.236486486486487,5.236486486486487, rmse=49.817567567567565,r2=-1.073507391783529, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5608108108108109, mae=5.128378378378378,5.128378378378378, rmse=51.38513513513514,r2=-0.9237106213082475, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.5540540540540541, mae=5.195945945945946,5.195945945945946, rmse=55.182432432432435,r2=-0.9960564906069171, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.49324324324324326, mae=5.1891891891891895,5.1891891891891895, rmse=52.445945945945944,r2=-0.8775175326341527, acc_naive=0.13513513513513514, mae_naive=4.614864864864865
pred: acc=0.527027027027027, mae=5.135135135135135,5.135135135135135, rmse=51.648648648648646,r2=-0.855793284

array([ 0.52733101,  5.15402404, 51.82642323, -0.89532198])

In [201]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

start pitlap: 28
--------- start update_onets: 28 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
--------- after update_onets: 28 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

--------- simu_onestep: 28-59: 28 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.65680003 3.98169994 4.02570009 4.12709999 4.31430006
 3.45009995 2.41529989 2.3900001  1.92139995 1.95550001]
[ 2.03548074  6.62483454 10.31511879  8.17620468  7.20253134  6.48333263
  5.79674578  5.1868329   4.81159163  4.49311924  4.20612478  3.9549284
  3.77134562  3.52572703  3.37624645  3.29536414  3.24495721  3.17235208
  4.8250556   4.5907917   3.74746156  3.44114923  2.94911194  2.51002526
  2.20196438  0.41787922  0.94617689  0.9147262   7.67670012  4.59170008]
--------- simu_onestep: 28-61: 28 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.6568000

--------- simu_onestep: 29-52: 29 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.65680003 3.98169994 4.02570009 4.12709999 4.31430006
 3.45009995 2.41529989 2.3900001  1.92139995 1.95550001 1.12639999]
[1.112867   1.16798258 1.17586815 1.2183423  5.72258472 9.68770123
 8.41980362 7.22516489 6.11906433 4.93001318 4.29879951 3.79879737
 3.60460329 3.52227449 3.35913157 3.13711882 2.87388802 4.2695055
 4.20900679 3.44589305 2.56410003 2.21609998]
--------- simu_onestep: 29-54: 29 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.65680003 3.98169994 4.02570009 4.12709999 4.31430006
 3.45009995 2.41529989 2.3900001  1.92139995 1.95550001 1.1263

--------- simu_onestep: 30-39: 30 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.65680003 3.98169994 4.02570009 4.12709999 4.31430006
 3.45009995 2.41529989 2.3900001  1.92139995 1.95550001 1.12639999
 1.60259998]
[1.71325195 1.59344053 1.48026884 1.40420771 1.35127378 5.58719254
 6.91590023 7.15500021]
--------- simu_onestep: 30-41: 30 ----------
[0.2113     0.4727     0.99989998 1.25139999 1.13979995 1.36590004
 1.73570001 1.98539996 1.90289998 1.98259997 1.90849996 1.92490005
 1.85099995 1.78799999 1.91190004 2.04570007 2.3822999  2.7026999
 3.11269999 3.65680003 3.98169994 4.02570009 4.12709999 4.31430006
 3.45009995 2.41529989 2.3900001  1.92139995 1.95550001 1.12639999
 1.60259998]
[1.71325195 1.59344053 1.48026884 1.40420771 1.35127378 5.58719254
 9.99250031 6.71449661 7.204      7.09130001]
--------- si

simulation done: 33
start pitlap: 31
--------- start update_onets: 31 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- after update_onets: 31 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

...

--------- simu_onestep: 167-174: 167 ----------
[ 0.2113      0.4727      0.99989998  1.25139999  1.13979995  1.36590004
  1.73570001  1.98539996  1.90289998  1.98259997  1.90849996  1.92490005
  1.85099995  1.78799999  1.91190004  2.04570007  2.3822999   2.7026999
  3.11269999  3.65680003  3.98169994  4.02570009  4.12709999  4.31430006
  3.45009995  2.41529989  2.3900001   1.92139995  1.95550001  1.12639999
  1.60259998 12.33539963 37.05250168 23.98240089  6.66590023  6.63660002
  6.71080017  6.91590023  7.15500021  7.204       7.09130001  6.62599993
  6.58059978  6.70550013  6.56120014  6.45230007  6.61380005 11.40869999
  6.58790016  2.66560006  2.56410003  2.21609998  2.1164      2.44510007
  0.33379999  2.76169991  4.42430019  7.67670012  4.59170008  3.92680001
  6.08720016  5.08309984  0.80809999  1.43929994  1.82770002  2.15680003
  2.60290003  4.92910004  3.48099995  1.7823      2.00889993  2.10599995
  0.37740001  0.59670001  0.7227      0.87260002  1.11230004  1.33860004
  1.

--------- simu_onestep: 167-182: 167 ----------
[ 0.2113      0.4727      0.99989998  1.25139999  1.13979995  1.36590004
  1.73570001  1.98539996  1.90289998  1.98259997  1.90849996  1.92490005
  1.85099995  1.78799999  1.91190004  2.04570007  2.3822999   2.7026999
  3.11269999  3.65680003  3.98169994  4.02570009  4.12709999  4.31430006
  3.45009995  2.41529989  2.3900001   1.92139995  1.95550001  1.12639999
  1.60259998 12.33539963 37.05250168 23.98240089  6.66590023  6.63660002
  6.71080017  6.91590023  7.15500021  7.204       7.09130001  6.62599993
  6.58059978  6.70550013  6.56120014  6.45230007  6.61380005 11.40869999
  6.58790016  2.66560006  2.56410003  2.21609998  2.1164      2.44510007
  0.33379999  2.76169991  4.42430019  7.67670012  4.59170008  3.92680001
  6.08720016  5.08309984  0.80809999  1.43929994  1.82770002  2.15680003
  2.60290003  4.92910004  3.48099995  1.7823      2.00889993  2.10599995
  0.37740001  0.59670001  0.7227      0.87260002  1.11230004  1.33860004
  1.

--------- simu_onestep: 167-190: 167 ----------
[ 0.2113      0.4727      0.99989998  1.25139999  1.13979995  1.36590004
  1.73570001  1.98539996  1.90289998  1.98259997  1.90849996  1.92490005
  1.85099995  1.78799999  1.91190004  2.04570007  2.3822999   2.7026999
  3.11269999  3.65680003  3.98169994  4.02570009  4.12709999  4.31430006
  3.45009995  2.41529989  2.3900001   1.92139995  1.95550001  1.12639999
  1.60259998 12.33539963 37.05250168 23.98240089  6.66590023  6.63660002
  6.71080017  6.91590023  7.15500021  7.204       7.09130001  6.62599993
  6.58059978  6.70550013  6.56120014  6.45230007  6.61380005 11.40869999
  6.58790016  2.66560006  2.56410003  2.21609998  2.1164      2.44510007
  0.33379999  2.76169991  4.42430019  7.67670012  4.59170008  3.92680001
  6.08720016  5.08309984  0.80809999  1.43929994  1.82770002  2.15680003
  2.60290003  4.92910004  3.48099995  1.7823      2.00889993  2.10599995
  0.37740001  0.59670001  0.7227      0.87260002  1.11230004  1.33860004
  1.

--------- simu_onestep: 167-198: 167 ----------
[ 0.2113      0.4727      0.99989998  1.25139999  1.13979995  1.36590004
  1.73570001  1.98539996  1.90289998  1.98259997  1.90849996  1.92490005
  1.85099995  1.78799999  1.91190004  2.04570007  2.3822999   2.7026999
  3.11269999  3.65680003  3.98169994  4.02570009  4.12709999  4.31430006
  3.45009995  2.41529989  2.3900001   1.92139995  1.95550001  1.12639999
  1.60259998 12.33539963 37.05250168 23.98240089  6.66590023  6.63660002
  6.71080017  6.91590023  7.15500021  7.204       7.09130001  6.62599993
  6.58059978  6.70550013  6.56120014  6.45230007  6.61380005 11.40869999
  6.58790016  2.66560006  2.56410003  2.21609998  2.1164      2.44510007
  0.33379999  2.76169991  4.42430019  7.67670012  4.59170008  3.92680001
  6.08720016  5.08309984  0.80809999  1.43929994  1.82770002  2.15680003
  2.60290003  4.92910004  3.48099995  1.7823      2.00889993  2.10599995
  0.37740001  0.59670001  0.7227      0.87260002  1.11230004  1.33860004
  1.

update lapstatus done.
simulation done: 0
start pitlap: 189
--------- start update_onets: 189 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- after update_onets: 189 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0

In [202]:
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
10,12,31,3.0,5.0,2.0,1,1.0,-2.0,-1
36,12,49,5.0,0.0,-5.0,-1,1.0,-4.0,-1
84,12,93,0.0,0.0,0.0,0,0.0,0.0,0
113,12,128,0.0,0.0,0.0,0,0.0,0.0,0


In [203]:
stint.get_evalret(df)

pred: acc=0.5238095238095238, mae=5.333333333333333,5.333333333333333, rmse=49.863945578231295,r2=-0.6086320409656183, acc_naive=0.1360544217687075, mae_naive=4.625850340136054


(0.5238095238095238,
 5.333333333333333,
 49.863945578231295,
 -0.6086320409656183)

In [148]:
acc = []
for i in ret.keys():
    df = ret[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

pred: acc=0.6688741721854304, mae=2.880794701986755,2.880794701986755, rmse=18.165562913907284,r2=0.14685223939623637, acc_naive=0.152317880794702, mae_naive=4.490066225165563
pred: acc=0.6556291390728477, mae=2.801324503311258,2.801324503311258, rmse=16.788079470198674,r2=0.22749912212369383, acc_naive=0.1456953642384106, mae_naive=4.516556291390729
pred: acc=0.6887417218543046, mae=2.814569536423841,2.814569536423841, rmse=17.70198675496689,r2=0.21604072626696613, acc_naive=0.16556291390728478, mae_naive=4.483443708609271
pred: acc=0.6556291390728477, mae=2.9403973509933774,2.9403973509933774, rmse=18.344370860927153,r2=0.24874272577052947, acc_naive=0.15894039735099338, mae_naive=4.496688741721854
pred: acc=0.6688741721854304, mae=2.9735099337748343,2.9735099337748343, rmse=19.370860927152318,r2=0.09289472711365232, acc_naive=0.152317880794702, mae_naive=4.503311258278146
pred: acc=0.6556291390728477, mae=2.8410596026490067,2.8410596026490067, rmse=17.56953642384106,r2=0.24227527558

array([ 0.67086093,  2.90397351, 18.65364238,  0.19329563])

In [145]:
a = [(1,2,3),(2,3,4)]
np.array(a)

array([[1, 2, 3],
       [2, 3, 4]])

In [147]:
len(ret)

10

In [149]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


In [150]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

a short ts: carid=88，len=4
start pitlap: 7
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 31
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 33
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 34
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 35
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 36
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 39
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 40
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 41
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 53
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 54
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 61


In [151]:
stint.get_evalret(df)

pred: acc=0.6704545454545454, mae=3.465909090909091,3.465909090909091, rmse=31.170454545454547,r2=0.1478118424583056, acc_naive=0.18181818181818182, mae_naive=4.568181818181818


(0.6704545454545454, 3.465909090909091, 31.170454545454547, 0.1478118424583056)

In [152]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

a short ts: carid=88，len=4
start pitlap: 7
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 31
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 33
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 34
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 35
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 36
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 39
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 40
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 41
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 53
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 54
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 61


simulation done: 30
start pitlap: 71
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 73
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 74
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 75
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 76
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 97
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 98
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 100
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 101
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 102
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 103
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 104
u

simulation done: 30
start pitlap: 123
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 128
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 131
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 132
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 133
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 134
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 135
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 136
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 138
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 139
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 140
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap

simulation done: 30
start pitlap: 170
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 171
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 172
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 173
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 174
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 175
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 176
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 177
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 178
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 179
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 180
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 

simulation done: 32
start pitlap: 36
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 39
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 40
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 41
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 53
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 54
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 61
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 63
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 64
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 65
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 66
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 31
start pitlap: 67
update

simulation done: 30
start pitlap: 97
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 98
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 100
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 101
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 102
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 103
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 104
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 105
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 106
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 107
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 108
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 

simulation done: 30
start pitlap: 134
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 135
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 136
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 138
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 139
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 140
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 141
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 143
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 144
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 145
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap: 164
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 30
start pitlap

simulation done: 26
start pitlap: 176
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 177
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 178
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 26
start pitlap: 179
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 180
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 181
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 183
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
start pitlap: 184
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 0
a short ts: carid=88，len=4
start pitlap: 7
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 31
update lapstatus done.
a short ts: carid=88，len=4
simulation done: 32
start pitlap: 33
update lapstatus done.
a short ts: carid=88，len=4
simulation don

In [153]:
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

pred: acc=0.6761363636363636, mae=3.4943181818181817,3.4943181818181817, rmse=32.17613636363637,r2=0.10486907867852768, acc_naive=0.18181818181818182, mae_naive=4.568181818181818
pred: acc=0.6818181818181818, mae=3.465909090909091,3.465909090909091, rmse=31.204545454545453,r2=0.10549772717177253, acc_naive=0.18181818181818182, mae_naive=4.568181818181818
pred: acc=0.6931818181818182, mae=3.4204545454545454,3.4204545454545454, rmse=31.136363636363637,r2=0.12889858706136936, acc_naive=0.18181818181818182, mae_naive=4.568181818181818
pred: acc=0.7272727272727273, mae=3.340909090909091,3.340909090909091, rmse=29.613636363636363,r2=0.15000426243240328, acc_naive=0.18181818181818182, mae_naive=4.568181818181818
pred: acc=0.6420454545454546, mae=3.6306818181818183,3.6306818181818183, rmse=32.78977272727273,r2=0.07122184598699877, acc_naive=0.18181818181818182, mae_naive=4.568181818181818
pred: acc=0.6704545454545454, mae=3.528409090909091,3.528409090909091, rmse=32.16477272727273,r2=0.1074489

array([ 0.68295455,  3.47159091, 31.42045455,  0.12311798])

### debug

In [160]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)


In [161]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

a short ts: carid=88，len=4
start pitlap: 7
--------- start update_onets: 7 ----------
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- after update_onets: 7 ----------
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.

--------- simu_onestep: 31 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 33 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

--------- simu_onestep: 35 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 37 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

--------- simu_onestep: 63 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 65 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

--------- simu_onestep: 49 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 51 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.

...

--------- simu_onestep: 188 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 190 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

--------- simu_onestep: 185 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 187 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

--------- simu_onestep: 183 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 185 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

--------- simu_onestep: 184 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 186 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

--------- simu_onestep: 187 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- simu_onestep: 189 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 

In [162]:
stint.get_evalret(df)

pred: acc=0.47368421052631576, mae=6.774436090225564,6.774436090225564, rmse=81.06015037593986,r2=-1.240454568182102, acc_naive=0.09022556390977443, mae_naive=5.684210526315789


(0.47368421052631576, 6.774436090225564, 81.06015037593986, -1.240454568182102)

### difference ? oracle .vs. pred

In [229]:
#df, df2
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,11.0,12.0,1.0,1,11.0,0.0,0
1,14,29,5.0,1.0,-4.0,-1,6.0,1.0,1
2,59,29,27.0,28.0,1.0,1,25.0,-2.0,-1
3,3,30,7.0,4.0,-3.0,-1,6.0,-1.0,-1
4,20,30,0.0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
146,25,156,18.0,0.0,-18.0,-1,14.0,-4.0,-1
147,64,156,6.0,3.0,-3.0,-1,5.0,-1.0,-1
148,59,157,21.0,19.0,-2.0,-1,19.0,-2.0,-1
149,60,157,16.0,1.0,-15.0,-1,3.0,-13.0,-1


In [230]:
df2

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,11.0,12.0,1.0,1,14.0,3.0,1
1,14,29,5.0,1.0,-4.0,-1,6.0,1.0,1
2,59,29,27.0,28.0,1.0,1,24.0,-3.0,-1
3,3,30,7.0,4.0,-3.0,-1,8.0,1.0,1
4,20,30,0.0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
142,25,156,18.0,0.0,-18.0,-1,16.0,-2.0,-1
143,64,156,6.0,3.0,-3.0,-1,19.0,13.0,1
144,59,157,21.0,19.0,-2.0,-1,20.0,-1.0,-1
145,60,157,16.0,1.0,-15.0,-1,16.0,0.0,0


In [231]:
oracle = df[['carno','startlap']].values
pred =  df2[['carno','startlap']].values

In [233]:
a = set(['%s-%s'%(x[0],x[1]) for x in oracle])
b = set(['%s-%s'%(x[0],x[1]) for x in  pred])
a -b

{'10-33', '3-127', '32-97', '33-30'}

In [234]:
df[df['carno']==3]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
3,3,30,7.0,4.0,-3.0,-1,6.0,-1.0,-1
34,3,49,4.0,2.0,-2.0,-1,4.0,0.0,0
85,3,93,2.0,4.0,2.0,1,2.0,0.0,0
115,3,127,4.0,4.0,0.0,0,1.0,-3.0,-1


In [235]:
df2[df2['carno']==3]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
3,3,30,7.0,4.0,-3.0,-1,8.0,1.0,1
32,3,49,4.0,2.0,-2.0,-1,4.0,0.0,0
83,3,93,2.0,4.0,2.0,1,0.0,-2.0,-1


In [236]:
_, data2018, _ = stint.load_data('Indy500-2018')
data2018[data2018['completed_laps']==200]

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


,rank,car_number,unique_id,completed_laps,elapsed_time,last_laptime,lap_status,best_laptime,best_lap,time_behind_leader,...,time_behind_prec,laps_behind_prec,overall_rank,overall_best_laptime,current_status,track_status,pit_stop_count,last_pitted_lap,start_position,laps_led
MyIdx,,,,,,,,,,,,,,,,,,,,,
18419,1,12,2,200,10782.6365,41.3651,T,40.3035,C7,0.0000,...,0.0000,0,1,40.3035,Active,K,5,AB,3,3B
18452,2,20,0,200,10785.7954,41.7902,T,40.2368,21,3.1589,...,3.1589,0,2,40.2368,Active,K,5,AD,1,41
18456,3,9,8,200,10787.2293,41.5713,T,40.6543,85,4.5928,...,1.4339,0,3,40.6543,Active,K,5,A0,9,0
18457,4,27,1F,200,10787.8602,41.5437,T,40.3275,B3,5.2237,...,0.6309,0,4,40.3275,Active,K,5,AE,20,1
18460,5,28,D,200,10789.3552,42.0466,T,40.4560,B4,6.7187,...,1.4950,0,5,40.4560,Active,K,5,AC,E,1
18461,6,22,1,200,10789.8722,42.2442,T,40.4933,B5,7.2357,...,0.5170,0,6,40.4933,Active,K,5,AF,2,1
18464,7,29,14,200,10790.4742,42.2352,T,40.1956,B2,7.8377,...,0.6020,0,7,40.1956,Active,K,6,B0,15,4
18466,8,1,3,200,10791.3282,41.7541,T,40.5625,AE,8.6917,...,0.8540,0,8,40.5625,Active,K,6,AA,4,3
18468,9,6,11,200,10791.9477,41.4786,T,40.5861,C6,9.3112,...,0.6195,0,9,40.5861,Active,K,7,BF,12,2


In [237]:
topK = data2018[data2018['completed_laps']==200]['car_number'].values
topK

array([12, 20,  9, 27, 28, 22, 29,  1,  6, 15, 66, 98,  4, 88, 25, 60, 64,
       23])

In [245]:
topK_df = df[df['carno'].isin(topK[:5])]
stint.get_evalret(topK_df)

pred: acc=0.55, mae=2.2,2.2, rmse=9.4,r2=0.08737864077669899, acc_naive=0.2, mae_naive=2.9


(0.55, 2.2, 9.4, 0.08737864077669899)

In [239]:
len(topK_df)

86

pred: acc=0.5813953488372093, mae=3.453488372093023,3.453488372093023, rmse=25.151162790697676,r2=-0.40560677043977655, acc_naive=0.16279069767441862, mae_naive=4.383720930232558


(0.5813953488372093,
 3.453488372093023,
 25.151162790697676,
 -0.40560677043977655)

In [244]:
topK_df2 = df2[df2['carno'].isin(topK[:5])]
stint.get_evalret(topK_df2)

pred: acc=0.5, mae=2.6,2.6, rmse=13.3,r2=-0.05639396346306613, acc_naive=0.2, mae_naive=2.9


(0.5, 2.6, 13.3, -0.05639396346306613)

In [246]:
len(topK_df2)

20